In [0]:
import csv
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchtext
import numpy as np
import matplotlib.pyplot as plt

In [0]:
import pandas

In [0]:
def get_data():
    header = ['tweet','label']
    data_set = pandas.read_csv('cleaned_data.txt',delimiter='\t',names = header)
    return data_set

# def get_label_mapping():
#     original_list = interesting_labels.wanted_list
#     output_list = [i for i in range(len(original_list))]
#     return dict(zip(original_list,output_list))
  
def split_tweet(tweet):
    return tweet.split()    

In [0]:
data_set = get_data()

In [0]:
'''
Create a vocabulary set that will contain all the uniue words in the tweets in our dataset
'''
all_words = []
list_of_list_of_tweets = []
for i in range(len(data_set['tweet'])):
    words = data_set['tweet'][i].split()
    list_of_list_of_tweets.append(words)
    for word in words:
        all_words.append(word)

In [6]:
len(list_of_list_of_tweets)
list_of_list_of_tweets[0]

['hoping', 'i', 'dont', 'screw', 'up', 'this', 'interview']

In [0]:
from collections import Counter

In [0]:
## Build a dictionary that maps words to integers
counts = Counter(all_words)
vocab = sorted(counts, key=counts.get, reverse=True)
vocab_to_int = {word: ii for ii, word in enumerate(vocab, 1)}


In [9]:
len(vocab)

62373

In [0]:
# ## use the dict to tokenize each review in reviews_split
# ## store the tokenized reviews in reviews_ints
tweets_ints = []
for tweet in list_of_list_of_tweets:
    tweets_ints.append([vocab_to_int[word] for word in tweet])

In [11]:
len(tweets_ints)

161893

In [0]:
encoded_labels = np.array(data_set['label'])

In [13]:
# outlier review stats
tweets_lens = Counter([len(x) for x in tweets_ints])
print("Zero-length tweets: {}".format(tweets_lens[0]))
print("Maximum tweet length: {}".format(max(tweets_lens)))

Zero-length tweets: 0
Maximum tweet length: 62


In [14]:
tweets_lens

Counter({1: 3340,
         2: 7871,
         3: 9817,
         4: 11462,
         5: 11761,
         6: 11545,
         7: 10978,
         8: 9911,
         9: 8730,
         10: 8037,
         11: 7391,
         12: 6781,
         13: 5946,
         14: 5389,
         15: 4935,
         16: 4642,
         17: 4190,
         18: 3768,
         19: 3585,
         20: 3189,
         21: 2961,
         22: 2723,
         23: 2384,
         24: 2283,
         25: 2004,
         26: 1690,
         27: 1479,
         28: 1073,
         29: 721,
         30: 523,
         31: 301,
         32: 206,
         33: 120,
         34: 63,
         35: 39,
         36: 25,
         37: 11,
         38: 6,
         39: 4,
         40: 2,
         41: 2,
         42: 1,
         43: 1,
         54: 1,
         62: 2})

In [15]:
for tweet in list_of_list_of_tweets:
    if len(tweet) == 62:
        print(" ".join(tweet))

you was way over there . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . and im only ickle not easy pushing through crowds
you was way over there . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . and im only ickle not easy pushing through crowds


In [0]:
def pad_features(tweets_ints, seq_length):
    ''' Return features of review_ints, where each review is padded with 0's 
        or truncated to the input seq_length.
    '''
    
    # getting the correct rows x cols shape
    features = np.zeros((len(tweets_ints), seq_length), dtype=int)

    # for each review, I grab that review and 
    for i, row in enumerate(tweets_ints):
        features[i, -len(row):] = np.array(row)[:seq_length]
    
    return features

In [0]:
seq_length = max(tweets_lens)

features = pad_features(tweets_ints, seq_length=seq_length)

In [18]:
print(features[:30,])

[[    0     0     0 ...    31    21  1681]
 [    0     0     0 ...    32  3291 33389]
 [    0     0     0 ...   137 33390   162]
 ...
 [    0     0     0 ...     4   726   159]
 [    0     0     0 ... 33399   504     1]
 [    0     0     0 ...   531    53  1726]]


### Train, Validation, Test Split 

In [0]:
from sklearn.utils import shuffle
features, encoded_labels = shuffle(features, encoded_labels)

In [41]:
split_frac = 0.8

## split data into training, validation, and test data (features and labels, x and y)

split_idx = int(len(features)*split_frac)
train_x, remaining_x = features[:split_idx], features[split_idx:]
train_y, remaining_y = encoded_labels[:split_idx], encoded_labels[split_idx:]

test_idx = int(len(remaining_x)*0.5)
val_x, test_x = remaining_x[:test_idx], remaining_x[test_idx:]
val_y, test_y = remaining_y[:test_idx], remaining_y[test_idx:]

## print out the shapes of your resultant feature data
print("\t\t\tFeature Shapes:")
print("Train set: \t\t{}".format(train_x.shape), 
      "\nValidation set: \t{}".format(val_x.shape),
      "\nTest set: \t\t{}".format(test_x.shape))

			Feature Shapes:
Train set: 		(129514, 62) 
Validation set: 	(16189, 62) 
Test set: 		(16190, 62)


### DataLoader and Batching

In [0]:
from torch.utils.data import TensorDataset, DataLoader

# create Tensor datasets
train_data = TensorDataset(torch.from_numpy(train_x), torch.from_numpy(train_y))
valid_data = TensorDataset(torch.from_numpy(val_x), torch.from_numpy(val_y))
test_data = TensorDataset(torch.from_numpy(test_x), torch.from_numpy(test_y))

# dataloaders
batch_size = 50

# make sure the SHUFFLE your training data
train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size,drop_last=True)
valid_loader = DataLoader(valid_data, shuffle=True, batch_size=batch_size,drop_last=True)
test_loader = DataLoader(test_data, shuffle=True, batch_size=batch_size,drop_last=True)

In [45]:
len(train_loader)

2590

### Defining the model

In [46]:
# First checking if GPU is available
train_on_gpu=torch.cuda.is_available()

if(train_on_gpu):
    print('Training on GPU.')
else:
    print('No GPU available, training on CPU.')

Training on GPU.


In [0]:
import torch.nn as nn

class SentimentRNN(nn.Module):
    """
    The RNN model that will be used to perform Sentiment analysis.
    """

    def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, n_layers, drop_prob=0.5):
        """
        Initialize the model by setting up the layers.
        """
        super(SentimentRNN, self).__init__()

        self.output_size = output_size
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim
        
        # embedding and LSTM layers
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, n_layers, 
                            dropout=drop_prob, batch_first=True)
        
        # dropout layer
        self.dropout = nn.Dropout(0.3)
        
        # linear and sigmoid layers
        self.fc = nn.Linear(hidden_dim, output_size)
        #self.sig = nn.Sigmoid()
        

    def forward(self, x, hidden):
        """
        Perform a forward pass of our model on some input and hidden state.
        """
        batch_size = x.size(0)

        # embeddings and lstm_out
        x = x.long()
        embeds = self.embedding(x)
        lstm_out, hidden = self.lstm(embeds, hidden)
    
        # stack up lstm outputs
        lstm_out = lstm_out.contiguous().view(-1, self.hidden_dim)
        
        # dropout and fully-connected layer
        out = self.dropout(lstm_out)
        out = self.fc(out)
        # sigmoid function
        #sig_out = self.sig(out)
        sig_out = out
        
        # reshape to be batch_size first
        sig_out = sig_out.view(batch_size, -1, output_size)
        sig_out = sig_out[:, -1] # get last batch of labels
        
        # return last sigmoid output and hidden state
        return sig_out, hidden
    
    
    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x hidden_dim,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_(),
                      weight.new(self.n_layers, batch_size, self.hidden_dim).zero_())
        
        return hidden
        

In [52]:
# Instantiate the model w/ hyperparams
vocab_size = len(vocab_to_int)+1 # +1 for the 0 padding + our word tokens
output_size = 6
embedding_dim = 400
hidden_dim = 256
n_layers = 1

net = SentimentRNN(vocab_size, output_size, embedding_dim, hidden_dim, n_layers)

print(net)

SentimentRNN(
  (embedding): Embedding(62374, 400)
  (lstm): LSTM(400, 256, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.3)
  (fc): Linear(in_features=256, out_features=6, bias=True)
)


/usr/local/lib/python3.6/dist-packages/torch/nn/modules/rnn.py:46: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


### Training

In [0]:
# loss and optimization functions
lr=0.0001

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=lr)

In [54]:
# training params

epochs = 2 # 3-4 is approx where I noticed the validation loss stop decreasing

counter = 0
print_every = 100
clip=5 # gradient clipping

# move model to GPU, if available
if(train_on_gpu):
    net.cuda()

net.train()
# train for some number of epochs
for e in range(epochs):
    # initialize hidden state
    h = net.init_hidden(batch_size)

    # batch loop
    for inputs, labels in train_loader:
        counter += 1

        if(train_on_gpu):
            inputs, labels = inputs.cuda(), labels.cuda()

        # Creating new variables for the hidden state, otherwise
        # we'd backprop through the entire training history
        h = tuple([each.data for each in h])

        # zero accumulated gradients
        net.zero_grad()

        # get the output from the model
        output, h = net(inputs, h)

        # calculate the loss and perform backprop
        loss = criterion(output, labels.long())
        loss.backward()
        # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
        nn.utils.clip_grad_norm_(net.parameters(), clip)
        optimizer.step()

        # loss stats
        if counter % print_every == 0:
            # Get validation loss
            val_h = net.init_hidden(batch_size)
            val_losses = []
            net.eval()
            for inputs, labels in valid_loader:

                # Creating new variables for the hidden state, otherwise
                # we'd backprop through the entire training history
                val_h = tuple([each.data for each in val_h])

                if(train_on_gpu):
                    inputs, labels = inputs.cuda(), labels.cuda()

                output, val_h = net(inputs, val_h)
                val_loss = criterion(output, labels.long())

                val_losses.append(val_loss.item())

            net.train()
            print("Epoch: {}/{}...".format(e+1, epochs),
                  "Step: {}...".format(counter),
                  "Loss: {:.6f}...".format(loss.item()),
                  "Val Loss: {:.6f}".format(np.mean(val_losses)))

Epoch: 1/2... Step: 100... Loss: 1.752846... Val Loss: 1.769815
Epoch: 1/2... Step: 200... Loss: 1.732317... Val Loss: 1.756974
Epoch: 1/2... Step: 300... Loss: 1.735572... Val Loss: 1.748361
Epoch: 1/2... Step: 400... Loss: 1.744648... Val Loss: 1.743155
Epoch: 1/2... Step: 500... Loss: 1.714126... Val Loss: 1.738031
Epoch: 1/2... Step: 600... Loss: 1.822322... Val Loss: 1.734642
Epoch: 1/2... Step: 700... Loss: 1.702513... Val Loss: 1.731913
Epoch: 1/2... Step: 800... Loss: 1.618994... Val Loss: 1.726707
Epoch: 1/2... Step: 900... Loss: 1.763071... Val Loss: 1.721139
Epoch: 1/2... Step: 1000... Loss: 1.731518... Val Loss: 1.717363
Epoch: 1/2... Step: 1100... Loss: 1.827555... Val Loss: 1.711233
Epoch: 1/2... Step: 1200... Loss: 1.634584... Val Loss: 1.710327
Epoch: 1/2... Step: 1300... Loss: 1.726228... Val Loss: 1.704936
Epoch: 1/2... Step: 1400... Loss: 1.698503... Val Loss: 1.702734
Epoch: 1/2... Step: 1500... Loss: 1.707241... Val Loss: 1.699783
Epoch: 1/2... Step: 1600... Loss: 

### Testing

In [57]:
# Get test data loss and accuracy

test_losses = [] # track loss
correct, total = 0, 0


# init hidden state
h = net.init_hidden(batch_size)

net.eval()
# iterate over test data
for inputs, labels in test_loader:

    # Creating new variables for the hidden state, otherwise
    # we'd backprop through the entire training history
    h = tuple([each.data for each in h])

    if(train_on_gpu):
        inputs, labels = inputs.cuda(), labels.cuda()
    
    # get predicted outputs
    output, h = net(inputs, h)
    
    # calculate loss
    test_loss = criterion(output, labels.long())
    test_losses.append(test_loss.item())
    
    # convert output probabilities to predicted class (0 or 1)
    pred = output.max(1, keepdim=True)[1]
    correct += pred.eq(labels.view_as(pred)).sum().item()
    total += labels.shape[0]
    # compare predictions to true label


# -- stats! -- ##
# avg test loss
print("Test loss: {:.3f}".format(np.mean(test_losses)))

# accuracy over all test data
test_acc = correct/total
print("Test accuracy: {:.3f}".format(test_acc))

Test loss: 1.649
Test accuracy: 0.309


### Inference

In [0]:
# negative test review
happy_tweet = 'Im so happy'
sad_tweet = 'I feel so down today'
angry = 'it boils my blood to see you'
surprised_tweet = 'wow what a nice car'
disgusted_tweet = 'im sick of this shit'
afraid_tweet = 'theres a stranger at my home'

In [114]:
def tokenize_review(test_tweet):
    # splitting by spaces
    test_words = test_tweet.lower().split()

    # tokens
    test_ints = []
    test_ints.append([vocab_to_int[word] for word in test_words])

    return test_ints

# test code and generate tokenized review
test_ints = tokenize_review(sad_tweet)
print(test_ints)

[[2, 105, 15, 158, 77]]


In [0]:
int_to_emotion = {0:'Happy', 1:'Sad' , 2:'Angry', 3:'Surprised', 4:'Disgusted', 5:'Afraid'}

def predict(net, test_tweet, sequence_length=max(tweets_lens)):
    
    net.eval()
    
    # tokenize review
    test_ints = tokenize_review(test_tweet)
    
    # pad tokenized sequence
    seq_length=sequence_length
    features = pad_features(test_ints, seq_length)
    
    # convert to tensor to pass into your model
    feature_tensor = torch.from_numpy(features)
    
    batch_size = feature_tensor.size(0)
    
    # initialize hidden state
    h = net.init_hidden(batch_size)
    
    if(train_on_gpu):
        feature_tensor = feature_tensor.cuda()
    
    # get the output from the model
    output, h = net(feature_tensor, h)
    
    # convert output probabilities to predicted class (0 or 1)
    pred = output.max(1, keepdim=True)[1] 
    # printing output value, before rounding
    print(test_tweet,' Prediction:', int_to_emotion[pred.item()])
    

In [122]:
predict(net,happy_tweet)
predict(net,sad_tweet)
predict(net,angry)
predict(net,surprised_tweet)
predict(net,disgusted_tweet)
predict(net,afraid_tweet)


Im so happy  Prediction: Happy
I feel so down today  Prediction: Sad
it boils my blood to see you  Prediction: Happy
wow what a nice car  Prediction: Surprised
im sick of this shit  Prediction: Disgusted
theres a stranger at my home  Prediction: Happy
